In [1]:
import numpy as np
from mofgbmlpy.fuzzy.knowledge.factory.homo_triangle_knowledge_factory_2_3_4_5 import HomoTriangleKnowledgeFactory_2_3_4_5
from mofgbmlpy.main.nsgaii.mofgbml_basic_main import MoFGBMLNSGAIIMain
import tracemalloc

cache_sizes = [0, 1000, 1000, 10000, 100000]
# cache_sizes = [100]
results = np.empty(len(cache_sizes), object)
ram_usage = np.empty(len(cache_sizes), object)
for i in range(len(cache_sizes)):
    print(f"#####\n# Step {i+1} / {len(cache_sizes)}")
    args = [
        # "--data-name", "iris",
        "--data-name", "pima",
        "--algorithm-id", "1",
        "--experiment-id", "2",
        # "--train-file", "../dataset/iris/a0_0_iris-10tra.dat",
        # "--test-file", "../dataset/iris/a0_0_iris-10tst.dat",
        "--train-file", "../dataset/pima/a0_0_pima-10tra.dat",
        "--test-file", "../dataset/pima/a0_0_pima-10tst.dat",        
        "--terminate-generation", "100",
        "--cache-size", f"{cache_sizes[i]}",
        "--objectives", "num-rules", "error-rate",
        # "--crossover-type", "pittsburgh-crossover",
        # "--antecedent-factory", "all-combination-antecedent-factory",
    ]
    tracemalloc.start()

    runner = MoFGBMLNSGAIIMain(HomoTriangleKnowledgeFactory_2_3_4_5)
    
    ram_usage[i] = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    results[i] = runner.main(args)

Execution times:

In [2]:
import matplotlib.pyplot as plt

ram_peak_usage_y_current = [r[0] for r in ram_usage]
ram_peak_usage_y_peak = [r[1] for r in ram_usage]
exec_time_y = [r.exec_time for r in results]

plt.plot(cache_sizes, ram_peak_usage_y_current)
plt.plot(cache_sizes, ram_peak_usage_y_peak)
plt.title("RAM usage")
plt.show()
plt.plot(cache_sizes, exec_time_y)
plt.title("Execution time")
plt.show()


Check hash function collisions

In [6]:
for i in range(1, len(cache_sizes)):
    if not np.array_equal(results[i-1].opt.get("X"), results[i].opt.get("X")):
        print(f"result {i-1} is not equal to result {i}")

In [7]:
arr1 = results[5].opt.get("X")[:,0]
arr2 = results[6].opt.get("X")[:,0]

for i in range(arr1.shape[0]):
    if arr1[i] != arr2[i]:
        print("diff solution")
        print(arr1[i])
        print(arr2[i])
        print("#####")
        for j in range(min(len(arr1[i].get_vars()), len(arr2[i].get_vars()))):
            if arr1[i].get_vars()[j] != arr2[i].get_vars()[j]:
                print("diff rules")
                print(arr1[i].get_vars()[j])
                print(arr2[i].get_vars()[j])
                print("#####")
                for k in range(min(len(arr1[i].get_vars()[j].get_vars()), len(arr2[i].get_vars()[j].get_vars()))):
                    if arr1[i].get_vars()[j].get_vars()[k] != arr2[i].get_vars()[j].get_vars()[k]:
                        print("diff antecedent")
        break

In [5]:
results[0].opt.get("X")

In [ ]:
results[1].opt.get("X")